# NYSE ML Project Machine Learning Analysis
In this notebook I will training and testing three different machine learning models with and without PCA.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%matplotlib inline

In [2]:
# Read in the data
cleaned_df = pd.read_csv('./data/cleaned/final-model.csv')

## Split Into Train and Test

In [3]:
from sklearn.model_selection import train_test_split

top_performing_raw = cleaned_df['95th Percentile']
features_raw = cleaned_df.drop(['95th Percentile'], axis=1)
symbols_raw = features_raw['Symbol']
features_raw = features_raw.drop(['Symbol'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(features_raw,
                                                    top_performing_raw,
                                                    test_size=0.2,
                                                    random_state=42)

print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


### Standardize Values

In [4]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

In [5]:
# Export scalar
from joblib import dump, load
dump(scalar, './models/scalar.joblib')

['./models/scalar.joblib']

### Naive Predictor Performance

In [6]:
true_positives = np.sum(top_performing_raw)
false_positives = top_performing_raw.count() - true_positives
true_negatives = 0
false_negatives = 0

accuracy = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)
fscore = (1 + pow(0.5,2)) * ((precision * recall) / ((pow(0.5,2) * precision) + recall))

# Setup time tracking
time_df = pd.DataFrame(columns=['Model', 'Time to Train', 'Time to Predict', 'F-score', 'Precision', 'Recall', 'Accuracy'])
new_row = {
    'Model': 'Naive Predictor',
    'Time to Train': 0,
    'Time to Predict': 0,
    'F-score': fscore,
    'Precision': precision,
    'Recall': recall,
    'Accuracy': accuracy
}

time_df.loc[len(time_df.index)] = new_row
time_df.head()

,Model,Time to Train,Time to Predict,F-score,Precision,Recall,Accuracy
0,Naive Predictor,0,0,0.066582,0.053985,1.0,0.053985


### Train Models

In [7]:
from sklearn.metrics import fbeta_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from pandas.core.frame import DataFrame
from pandas.core.series import Series
import timeit

# Function to train predict, and populate scores in the results dictionary
def train_predict(model:str,
                  grid_obj:GridSearchCV ,
                  x_train: DataFrame,
                  y_train: DataFrame,
                  x_test: Series,
                  y_test: Series) -> dict: 
    '''
    inputs:
        - model: the name of the model
        - grid_obj: the grid search object to train and predict
        - x_train: features training set
        - y_train: income training set
        - x_test: features testing set
        - y_test: income testing set
    '''
    
    results = {}
    results['Model'] = model
    
    start = timeit.default_timer()
    grid_fit = grid_obj.fit(x_train, y_train)
    end = timeit.default_timer()
    learner = grid_fit.best_estimator_
    
    results['Time to Train'] = end - start
        
    start = timeit.default_timer()
    predictions_test = learner.predict(x_test)
    end = timeit.default_timer()
    
    results['Time to Predict'] = end - start
            
    results['F-score'] = fbeta_score(y_test, predictions_test, beta=0.5, average='micro')
        
    results['Precision'] = precision_score(y_test, predictions_test, average='micro')
    
    results['Recall'] = recall_score(y_test, predictions_test, average='micro')
        
    results['Accuracy'] = accuracy_score(y_test, predictions_test)
       
    return results, learner


#### Without PCA
##### SVC

In [8]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc_no_pca = SVC(random_state=42)

scorer = make_scorer(fbeta_score, beta=0.5, average='micro')
parameters = {'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
              'shrinking': [True, False],
              'probability': [True, False],
              'decision_function_shape': ['ovo', 'ovr']}

grid_obj = GridSearchCV(svc_no_pca, parameters, scoring=scorer)

results, svc_no_pca = train_predict('SVC Without PCA', grid_obj, x_train, y_train, x_test, y_test)
time_df.loc[len(time_df.index)] = results
display(time_df.head())

print(svc_no_pca.get_params())
dump(svc_no_pca, './models/svc_no_pca.joblib')

,Model,Time to Train,Time to Predict,F-score,Precision,Recall,Accuracy
0,Naive Predictor,0.000000,0.000000,0.066582,0.053985,1.000000,0.053985
1,SVC Without PCA,0.915877,0.000355,0.974359,0.974359,0.974359,0.974359


{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovo', 'degree': 3, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'probability': True, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}


['./models/svc_no_pca.joblib']